###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2017 L.A. Barba, N.C. Clementi

# Catch things in motion

This module of the _Engineering Computations_ course is our launching pad to investigate _change_, _motion_, _dynamics_, using computational thinking, Python, and Jupyter.

The foundation of physics and engineering is the subject of **mechanics**: how things move around, when pushed around. Or pulled… in the beginning of the history of mechanics, Galileo and Newton seeked to understand how and why objects fall under gravity.

This first lesson will explore motion by analyzing images and video, to learn about velocity and acceleration.

## Acceleration of a fallling ball

Let's start at the beginning. Suppose you want to use video capture of a falling ball to _compute_ the acceleration of gravity. Could you do it? With Python, of course you can!

Here is a neat video we found online, produced over at MIT several years ago. It shows a ball being dropped in front of a metered panel, while lit by a stroboscopic light. Watch the video!

In [ ]:
from IPython.display import YouTubeVideo
vid = YouTubeVideo("xQ4znShlK5A")
display(vid)

We learn on the video that the marks on the panel are every $0.25\rm{m}$, and on the [website](http://techtv.mit.edu/collections/physicsdemos/videos/831-strobe-of-a-falling-ball) they say that the strobe light flashes at about 15 Hz (that's 15 times per second). The final [image on Flickr](https://www.flickr.com/photos/physicsdemos/3174207211), however, notes that the strobe fired 16.8 times per second. So we have some uncertainty already!

Luckily, the MIT team obtained one frame with the ball visible at several positions as it falls. This, thanks to the strobe light and a long-enough exposure of that frame. What we'd like to do is use that frame to capture the ball positions digitally, and then obtain the velocity and acceleration from the distance over time. 

You can find several toolkits for handling images and video with Python; we'll start with a simple one called [`imageio`](https://imageio.github.io). Import this library like any other, and let's load `numpy` and `pyplot` while we're at it.


In [ ]:
import imageio
import numpy
from matplotlib import pyplot

### Read the video

With the `get_reader()` method of `imageio`, you can read a video from its source into a _Reader_ object. You don't need to worry too much about the technicalities here—we'll walk you through it all—but check the type, the length (for a video, that's number of frames), and notice you can get info, like the frames-per-second, using `get_meta_data()`.

In [ ]:
reader = imageio.get_reader('http://techtv.mit.edu/videos/831-strobe-of-a-falling-ball/download.mp4')

In [ ]:
type(reader)

In [ ]:
len(reader)

In [ ]:
fps = reader.get_meta_data()['fps']
print(fps)

##### Note:

You may get this error after calling `get_reader()`:

NeedDownloadError: Need ffmpeg exe. You can obtain it with either:
  - install using conda: `conda install ffmpeg -c conda-forge`
  - download by calling: `imageio.plugins.ffmpeg.download()`

If you do, follow the tips to install the needed `ffmpeg` tool.

### Capture mouse clicks on a frame

With `imageio`, you can grab one frame of the video, and then use `pyplot` to show it as an image. But we want to interact with the image, somehow.

So far in this course, we have used the command `%matplotlib inline` to get our plots rendered _inline_ in a Jupyter notebook. There is an alternative command that gives you some interactivity on the figures: `%matplotlib notebook`. Execute this now, and you'll see what it does below.

In [ ]:
%matplotlib notebook

Now we can use the `get_data()` method on the `imageio` _Reader_ object, to grab one of the video frames, passing the frame number. Below, we use it to grab frame number 1100, and then use the `shape` attribute to see that it is an "array-like" object with three dimensions: they are the pixel numbers in the horizontal and vertical directions, and the number of colors (3 colors in RGB format). Check the type to see that it's an `imageio` _Image_ object.

In [ ]:
image = reader.get_data(1100)
image.shape

In [ ]:
type(image)

Naturally, `imageio` plays well with `pyplot`. You can use [`pyplot.imshow`](https://matplotlib.org/devdocs/api/_as_gen/matplotlib.pyplot.imshow.html) to show the image in a figure. 

Check out the neat interactive options that we get with `%matplotlib notebook`. Then you can go back and change the frame number above, and show it below. We chose frame 1100 after playing around a bit and finding that it gives a good view of the long-exposure image of the falling ball.

In [ ]:
pyplot.imshow(image, interpolation='nearest');

Okay! Here is where things get really interesting. Matplotlib has the ability to create [event connections](https://matplotlib.org/devdocs/users/event_handling.html?highlight=mpl_connect), that is, conenct the figure canvas to user-interface events on it, like mouse clicks. 

To use this ability, you write a function with the events you want to capture, and then connect this function to the Matplotlib "event manager" using [`mpl_connect()`](https://matplotlib.org/devdocs/api/backend_bases_api.html#matplotlib.backend_bases.FigureCanvasBase.mpl_connect). In this case, we connect the `'button_press_event'` to the function named `onclick`, which captures the $(x,y)$ coordinates of the mouse click on the figure. Magic!

In [ ]:
fig = pyplot.figure()

pyplot.imshow(image, interpolation='nearest')

coords = []
def onclick(event):
    '''Capture the x,y coordinates of a mouse click on the image'''
    ix, iy = event.xdata, event.ydata
    coords.append([ix, iy]) 

connectId = fig.canvas.mpl_connect('button_press_event', onclick)


Notice that in the code cell above, we created an empty list named `coords`, and inside the `onclick()` function, we are appending the $(x,y)$ coordinates of each mouse click on the figure. After executing the cell above, you have a connection to the figure, via the user interface: try clicking with your mouse on a few white lines of the metered panel (on the edge of the panel, to get approximately equal $x$ coordinate), then print the contents of the `coords` list below.

In [ ]:
coords

The $x$ coordinates are pretty close, but there is some variation due to our shaky hand (or bad eyesigh), and perhaps because the metered panel is not perfectly vertical. We can cast the list to a NumPy array, then grab all the first elements of the coordinate pairs, then get the standard deviation as an indication of our error in the mouse-click captures.

In [ ]:
numpy.array(coords)[:,0]

In [ ]:
numpy.array(coords)[:,0].std()

Depending how shaky _your_ hand was, you may get a different value, but we got a standard deviation of about one pixel. Pretty good!

Now, let's grab all the second elements of the coordinate pairs, corresponding to the $y$ coordinate, i.e., the ball's vertical position on the video frame.

In [ ]:
y_lines = numpy.array(coords)[:,1]
y_lines

What we need, though, is to have this vertical position in physical length (in meters), not in the pixel number on the image. Knowing that the distance between two white lines on the metered panel is $0.25\rm{m}$ will help. Let's get the average vertical distance between two lines, which we can calculate from:

\begin{equation}
\overline{\Delta y} = \sum \frac{y_{i+1}-y_i}{N-1}
\end{equation}

In [ ]:
gap_lines = y_lines[1:] - y_lines[0:-1]
gap_lines.mean()

In [ ]:
fig = pyplot.figure()

pyplot.imshow(image, interpolation='nearest')

coords = []
def onclick(event):
    '''Capture the x,y coordinates of a mouse click on the image'''
    ix, iy = event.xdata, event.ydata
    coords.append([ix, iy]) 

connectId = fig.canvas.mpl_connect('button_press_event', onclick)

In [ ]:
coords

In [ ]:
y_coords = numpy.array(coords)[:,1]
delta_y = (y_coords[1:] - y_coords[:-1]) *0.25 / gap_lines.mean()

\begin{equation}
\bar{v}_i = \frac{y_{i+1}-y_i}{\Delta t}
\end{equation}

In [ ]:
v = delta_y * 16.8
v

In [ ]:
a = (v[1:] - v[:-1]) *16.8
a

In [ ]:
a[1:].mean()

In [ ]:
a-9.81

In [ ]:
pyplot.figure()
pyplot.plot(a-9.81, ':x')
pyplot.show()

### Capture clicks from movie frames

We'd like to capture the coordinates of mouse clicks on a sequence of images, so that we may have the positions of a moving ball caught on video. We know how to capture the coordinates of mouse clicks, so the challenge is to get consecutive frames of the video displayed for us, to click on the ball position each time. 

Widgets to the rescue! There are currently [10 different widget types](http://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html) included in the `ipywidgets` library. The `BoundedIntText()` widget shows a text box with an integer value that can be stepped from a minimum to a maximum value by clicking up/down arrows. Stepping through frames with this widget, and clicking on the ball position each time, gets us what we want.

Digitizing the ball positions in this way is a bit tedious. But this could be a realistic scenario: you captured video of a moving object, and you need to get position data from the video frames. Unless you have some fancy motion-capture equipment, this will do the trick.

Let's load the Jupyter widgets:

In [ ]:
reader = imageio.get_reader('Sample.mp4')

In [ ]:
fps = reader.get_meta_data()['fps']
print(fps)

In [ ]:
image = reader.get_data(0)

In [ ]:
image.shape

In [ ]:
type(image)

In [ ]:
image1 = reader.get_data(10)
image2 = reader.get_data(20)
pyplot.imshow(image1, interpolation='nearest', alpha=0.9)
pyplot.imshow(image2, interpolation='nearest', alpha=0.7)


**Notes**

`iter_data()`
Iterate over all images in the series. (Note: you can also iterate over the reader object.)

Iterate over frames in a movie:

```Python
for i, im in enumerate(reader):```

In [ ]:
from ipywidgets import widgets

In [ ]:
selector = widgets.BoundedIntText(value=0, min=0, max=38, step=1,
    description='Frame:',
    disabled=False)

coords = []
def onclick(event):
    '''Capture the x,y coordinates of a mouse click on the image'''
    ix, iy = event.xdata, event.ydata
    coords.append([ix, iy]) 


def catchclick(frame):
    image = reader.get_data(frame)
    pyplot.imshow(image, interpolation='nearest');



fig = pyplot.figure()
#fig.add_subplot()

connectId = fig.canvas.mpl_connect('button_press_event', onclick)

widgets.interact(catchclick, frame=selector);

In [ ]:
coords

In [ ]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../../style/custom.css'
HTML(open(css_file, "r").read())